### 0. Initial Setting

In [1]:
# %%capture
# !pip install datasets==1.0.2
# !pip install transformers==4.2.1

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

cache_dir = "/data4/yoomcache"
model_cache_dir = os.path.join(cache_dir, 'huggingface')
data_cache_dir = os.path.join(cache_dir, 'datasets')
checkpoint_dir = os.path.join(cache_dir, 'checkpoint2')

import logging
logging.getLogger().setLevel(logging.CRITICAL)
logging.basicConfig(level=logging.INFO)


import torch
from datasets import load_dataset, load_metric, load_from_disk
from transformers import BertTokenizer, RobertaTokenizer, GPT2Tokenizer
from transformers import AutoConfig, EncoderDecoderConfig, EncoderDecoderModel
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

import wandb
wandb.init(project="testing-roberta2gpt", entity="yoom-private")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yoom-private (use `wandb login --relogin` to force relogin)


### 1. Initialize Model

In [3]:
config_encoder = AutoConfig.from_pretrained("roberta-base", cache_dir=model_cache_dir)
config_decoder = AutoConfig.from_pretrained("gpt2", cache_dir=model_cache_dir)
config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder, cache_dir=model_cache_dir)
model = EncoderDecoderModel(config=config)
# model.save_pretrained("roberta2gpt", cache_dir=model_cache_dir)
# model = EncoderDecoderModel.from_pretrained("roberta2gpt", cache_dir=model_cache_dir)

model.encoder.encoder.layer = model.encoder.encoder.layer[:8]
model.decoder.transformer.h = model.decoder.transformer.h[-8:]

In [4]:
encoder_tokenizer = RobertaTokenizer.from_pretrained("roberta-base", cache_dir=model_cache_dir)
encoder_tokenizer.bos_token = encoder_tokenizer.cls_token  # CLS token will work as BOS token
encoder_tokenizer.eos_token = encoder_tokenizer.sep_token  # SEP token will work as EOS token

# make sure GPT2 appends EOS in begin and end
def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
    return outputs

GPT2Tokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens
decoder_tokenizer = GPT2Tokenizer.from_pretrained("gpt2", cache_dir=model_cache_dir)
# set pad_token_id to unk_token_id -> be careful here as unk_token_id == eos_token_id == bos_token_id
decoder_tokenizer.pad_token = decoder_tokenizer.unk_token


model.config.decoder_start_token_id = encoder_tokenizer.cls_token_id
model.config.eos_token_id = encoder_tokenizer.sep_token_id
model.config.pad_token_id = encoder_tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size


# set decoding params
model.config.decoder_start_token_id = decoder_tokenizer.bos_token_id
model.config.eos_token_id = decoder_tokenizer.eos_token_id
model.config.max_length = 142
model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.early_stopping = True
model.length_penalty = 2.0
model.num_beams = 4

In [5]:
# Freeze encoder parameters
for param in model.encoder.parameters():
    param.requires_grad = False
with torch.no_grad():
    model.encoder
    
# # Freeze decoder parameters
# for param in model.encoder.parameters():
#     param.requires_grad = False

### 2. Preparing Dataset

In [6]:
# map data correctly
def map_to_encoder_decoder_inputs(batch):    # Tokenizer will automatically set [BOS] <text> [EOS] 
    encoder_length, decoder_length = 512, 128
    inputs = encoder_tokenizer(batch["article"], 
                               padding="max_length", 
                               truncation=True, 
                               max_length=encoder_length)
    outputs = decoder_tokenizer(batch["highlights"], 
                                padding="max_length", 
                                truncation=True, 
                                max_length=decoder_length)
    
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = -100 if batch["decoder_attention_mask"] == 0 else batch["labels"]

    assert len(inputs.input_ids) == encoder_length
    assert len(outputs.input_ids) == decoder_length

    return batch

In [7]:
if os.path.exists(os.path.join(cache_dir, 'preprocessed/train')):
    train_dataset = load_from_disk(os.path.join(cache_dir, 'preprocessed/train'))
else:
    train_dataset = load_dataset("ccdv/cnn_dailymail", "3.0.0", split="train", cache_dir=data_cache_dir)
    train_dataset = train_dataset.map(
        map_to_encoder_decoder_inputs, 
        # batched=True, 
        # batch_size=batch_size, 
        remove_columns=['id', 'article', 'highlights'],
    )
    train_dataset.set_format(
        type="torch", 
        columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
    )
    
    train_dataset.save_to_disk(os.path.join(cache_dir, 'preprocessed/train'))


if os.path.exists(os.path.join(cache_dir, 'preprocessed/val')):
    val_dataset = load_from_disk(os.path.join(cache_dir, 'preprocessed/val'))
else:
    val_dataset = load_dataset("ccdv/cnn_dailymail", "3.0.0", split="validation", cache_dir=data_cache_dir)
    val_dataset = val_dataset.map(
        map_to_encoder_decoder_inputs, 
        # batched=True, 
        # batch_size=batch_size, 
        remove_columns=['id', 'article', 'highlights'],
    )
    val_dataset.set_format(
        type="torch", 
        columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
    )
    val_dataset.save_to_disk(os.path.join(cache_dir, 'preprocessed/val'))

### Training Model

In [8]:
# load rouge for validation
rouge = load_metric("rouge")
# rouge = load_metric("rouge", experiment_id=1)

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = decoder_tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = decoder_tokenizer.eos_token_id
    label_str = decoder_tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
batch_size = 16

# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    output_dir=os.path.join(checkpoint_dir, "roberta2gpt"),
    do_train=True,
    do_eval=True,
    # do_predict=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=1e-4, 
#     weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0,
    num_train_epochs=100,
    max_steps=-1,
    lr_scheduler_type='cosine',
    
    logging_strategy='steps',
    save_strategy='steps',
    evaluation_strategy='steps',
    logging_steps=1000,
    save_steps=1000,
    eval_steps=1000,
    warmup_steps=10000,
    save_total_limit=10,
    overwrite_output_dir=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# start training
trainer.train()

/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 28711
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 179500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You ma

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
1000,2.468400,0.363385,0.000000,0.000100,0.000100
2000,0.347100,0.012939,0.000000,0.000000,0.000000
3000,0.025700,0.001739,0.000000,0.000000,0.000000
4000,0.005600,0.000756,0.000000,0.000000,0.000000
5000,0.002200,0.000445,0.000000,0.000000,0.000000
6000,0.001100,0.000285,0.000000,0.000100,0.000000
7000,0.000600,0.000213,0.000000,0.000000,0.000000
8000,0.000300,0.000165,0.000000,0.000000,0.000000
9000,0.000200,0.000138,0.000000,0.000000,0.000000
10000,0.000100,0.000119,0.000000,0.000100,0.000000


***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-1000
Configuration saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-1000/config.json
Model weights saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-88000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION

***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-9000
Configuration saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-9000/config.json
Model weights saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-9000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-96000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION

***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-17000
Configuration saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-17000/config.json
Model weights saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-17000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-7000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATI

...

***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-33000
Configuration saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-33000/config.json
Model weights saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-33000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-23000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECAT

***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-41000
Configuration saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-41000/config.json
Model weights saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-41000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-31000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECAT

***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-49000
Configuration saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-49000/config.json
Model weights saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-49000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-39000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECAT

***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-57000
Configuration saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-57000/config.json
Model weights saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-57000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-47000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECAT

***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-65000
Configuration saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-65000/config.json
Model weights saved in /data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-65000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint2/roberta2gpt/checkpoint-55000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECAT